# Main class for running the molecule environment

In [1]:
import gym
from gym import wrappers, logger
from MoleculeAgent import MoleculeAgent
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
import matplotlib.pyplot as plt
# TODO: Render as a main function too
# Render toggled in the main class 
# find the correct time delay
# 
# Add pop in modifications

In [2]:
def isValid(mol):
    try:
        molecule = Chem.MolFromSmiles(mol)
        smiles = Chem.MolToSmiles(molecule, isomericSmiles=True)
    except:
        print("Entered input is chemically invalid")
        return False
    else:
        return True


def main():
    print("Welcome to TEDD, the Training Environment for Drug Discovery.")
    print("In order to run the environment, you will need to input a starting molecule, a target molecule and an optimisation goal.")
  
    # determine starting molecule
    while True:
        print("Step 1) Choose starting molecule: \nInput ‘1’ to choose a CARBON molecule.\nInput ‘2’ to choose a RANDOM molecule.")
        starting_molecule = input("Input: ")
        if not(starting_molecule == "1" or starting_molecule == "2"):
            print("Incorrect Input.\n")
        elif starting_molecule == '1':
            starting_molecule = "C" # get carbon molecule 
            break
        else:
            starting_molecule = "1" # get random molecule 
            break

    # determine target molecule
    while True:
        print("\nStep 2) Choose target molecule: \nInput ‘1’ to SPECIFY a molecule.\nInput ‘2’ to choose a RANDOM molecule.")
        target_molecule = input("Input: ")
        if not(target_molecule == '1' or target_molecule == '2'):
            print("Incorrect Input.\n")
        elif target_molecule == '1':
            print("Input the target molecule.")
            target_molecule = input("Input: ")
            if isValid(target_molecule): # check validity of molecule (?)
                break
            else:
                print("Incorrect Input.\n")
        else:
            target_molecule = "1" # get random molecule 
            break

    # determine optimisation goal
    while True:
        print("\nStep 3) Choose optimisation goal: \nInput a floating-point value between 0 and 1.")
        optimisation_goal = input("Input: ")
        try:
            optimisation_goal = float(optimisation_goal)
            if optimisation_goal < 0 or optimisation_goal > 100:
                print("Incorrect Input.\n")
            else:
                break
        except:
            print("Incorrect Input.\n")

    env = gym.make("gym_molecule:molecule-v0", start = starting_molecule, target = target_molecule, goal = optimisation_goal)
                
    print("Environment initialiased.")
     
    agent = MoleculeAgent(env.observation_space, env.action_space) # Create a new Molecule Agent object.
    
    
    episode_count = 2
    reward = 0
    done = False

    
    
    for episode in range(1,5):

        state = env.reset()
        print("Episode: " + str(episode) + ", Starting State: " + str(state))

        for step in range(1,21):
            action = agent.act(state, reward, done)
            print("Action: ", action) 

            state_next, reward, done, info = env.step(action)
            print("State:", state_next, "\nReward:", reward, "\nDone:", done,"\nInfo: ")

            # agent.remember(state, action, reward, state_next, done)
            state = state_next

            env.render()

            if done:
                print("Optimisation Goal Achieved - Episode: " + str(episode) + ", Step: " + str(step) + ", Similarity %: " + str(state))
                break  
        
    env.close()
            
    


        
    
    
if __name__ == '__main__':
    main()

# Aspirin: O=C(C)Oc1ccccc1C(=O)O
# Start with: Oc1ccccc1C

Welcome to TEDD, the Training Environment for Drug Discovery.
In order to run the environment, you will need to input a starting molecule, a target molecule and an optimisation goal.
Step 1) Choose starting molecule: 
Input ‘1’ to choose a CARBON molecule.
Input ‘2’ to choose a RANDOM molecule.
Input: 1

Step 2) Choose target molecule: 
Input ‘1’ to SPECIFY a molecule.
Input ‘2’ to choose a RANDOM molecule.
Input: 2

Step 3) Choose optimisation goal: 
Input a floating-point value between 0 and 1.
Input: 45


SyntaxError: invalid syntax (Chemistry.py, line 241)

## 